In [25]:
import pandas as pd
import numpy as np
import plotly.express as px
import math
import plotly.graph_objects as go


url = 'https://raw.githubusercontent.com/entzyeung/towardsdatascience/main/01_%20satellite/satellites.csv'
data = pd.read_csv(url, index_col=False, dtype={"satelliteA_ID":str})
data.sample(5)

,satelliteA_ID,satelliteA_Ratings,satelliteA_latitude,satelliteA_longitude,satelliteB_latitude,satelliteB_longitude
216,4AC4,4.9,18.533811,73.899315,18.623811,73.989315
548,C107,4.1,-22.515082,88.367830,22.645082,88.497830
368,C1B4,4.7,9.988483,76.295211,10.048483,76.355211
101,4460,4.9,18.536562,73.896485,18.556562,73.916485
649,C007,4.6,-23.264015,-77.408236,23.294015,77.438236


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   satelliteA_ID         657 non-null    object 
 1   satelliteA_Ratings    657 non-null    float64
 2   satelliteA_latitude   657 non-null    float64
 3   satelliteA_longitude  657 non-null    float64
 4   satelliteB_latitude   657 non-null    float64
 5   satelliteB_longitude  657 non-null    float64
dtypes: float64(5), object(1)
memory usage: 30.9+ KB


In [27]:
data.isnull().sum()

satelliteA_ID           0
satelliteA_Ratings      0
satelliteA_latitude     0
satelliteA_longitude    0
satelliteB_latitude     0
satelliteB_longitude    0
dtype: int64

# Visualize all the satellites on map

In [28]:
# visualize all the all satellites A B on map

fig1 = px.scatter_geo(data,lat='satelliteA_latitude',lon='satelliteA_longitude', 
                      locationmode="country names",
                      size_max = 100, 
                      projection= "natural earth")
fig2 = px.scatter_geo(data,lat='satelliteB_latitude',lon='satelliteB_longitude',
                      locationmode="country names",
                      size_max = 100, 
                      projection= "natural earth")

fig=px.scatter_geo()
fig.add_traces(fig1._data)
fig.add_traces(fig2._data)

fig.data[0].marker.color = 'rgb(0, 128, 0)'
fig.data[1].marker.color = 'rgb(255, 0, 0)'
fig.update_layout(
    title='All Satellite Positions',
    geo=dict(
        resolution=110,
        showland=True,
        showlakes=True,
        landcolor='rgb(204, 204, 204)',
        countrycolor='rgb(204, 204, 204)',
        lakecolor='rgb(255, 255, 255)',
        projection_type='equirectangular',
        coastlinewidth=0.5,
        lataxis=dict(range=[-90, 90]),
        lonaxis=dict(range=[-180, 180])
    )
)

fig.show()

# Calculates the distance between two points on the Earth's surface

In [29]:
# This code calculates the distance between two points on the Earth's surface using the Haversine formula.
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert latitude and longitude to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # The differences in longitude and latitude,
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # a represents the square of half the chord length between the points,
    # c represents the angular distance in radians.
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance
  

In [30]:
# Create distance column & calculate the distance between each pair of points
data['distance'] = np.nan

for i in range(len(data)):
  data.loc[i, 'distance'] = calculate_distance(data.loc[i, 'satelliteA_latitude'], 
                                          data.loc[i, 'satelliteA_longitude'], 
                                          data.loc[i, 'satelliteB_latitude'], 
                                          data.loc[i, 'satelliteB_longitude'])

In [31]:
data.head()

,satelliteA_ID,satelliteA_Ratings,satelliteA_latitude,satelliteA_longitude,satelliteB_latitude,satelliteB_longitude,distance
0,4607,4.9,22.745049,75.892471,22.765049,75.912471,3.025149
1,B379,4.5,12.913041,77.683237,13.043041,77.813237,20.183530
2,5D6D,4.4,12.914264,77.678400,12.924264,77.688400,1.552758
3,7A6A,4.7,11.003669,76.976494,11.053669,77.026494,7.790401
4,70A2,4.6,12.972793,80.249982,13.012793,80.289982,6.210138


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   satelliteA_ID         657 non-null    object 
 1   satelliteA_Ratings    657 non-null    float64
 2   satelliteA_latitude   657 non-null    float64
 3   satelliteA_longitude  657 non-null    float64
 4   satelliteB_latitude   657 non-null    float64
 5   satelliteB_longitude  657 non-null    float64
 6   distance              657 non-null    float64
dtypes: float64(6), object(1)
memory usage: 36.1+ KB


# Visualizing Satellite Pairs Individually

In [33]:
# Coordinates of satellite A above London
satellite_A_lat = 51.5074
satellite_A_lon = -0.1278

# Coordinates of satellite B above Brighton
satellite_B_lat = 50.8225
satellite_B_lon = -0.1372

In [34]:
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert latitude and longitude to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance

distance = calculate_distance(satellite_A_lat, satellite_A_lon, satellite_B_lat, satellite_B_lon)
print(f"The distance between London and Brighton is approximately {distance:.2f} kilometers.")


The distance between London and Brighton is approximately 76.16 kilometers.


In [35]:
# Create a scatter_geo plot
fig = go.Figure()

# Add satellite A
fig.add_trace(go.Scattergeo(
    lon=[satellite_A_lon],
    lat=[satellite_A_lat],
    mode='markers',
    marker=dict(color='blue', size=8),
    name='Satellite A'
))

# Add satellite B
fig.add_trace(go.Scattergeo(
    lon=[satellite_B_lon],
    lat=[satellite_B_lat],
    mode='markers',
    marker=dict(color='red', size=8),
    name='Satellite B'
))

# Add line connecting the satellites
fig.add_trace(go.Scattergeo(
    lon=[satellite_A_lon, satellite_B_lon],
    lat=[satellite_A_lat, satellite_B_lat],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Distance'
    
))

# Set layout properties
fig.update_layout(
    title='Satellite Positions',
    geo=dict(
        resolution=110,
        showland=True,
        showlakes=True,
        landcolor='rgb(204, 204, 204)',
        countrycolor='rgb(204, 204, 204)',
        lakecolor='rgb(255, 255, 255)',
        projection_type='equirectangular',
        coastlinewidth=0.5,
        lataxis=dict(range=[-90, 90]),
        lonaxis=dict(range=[-180, 180])
    )
)

# Add a scatter trace with text annotation for the distance number
fig.add_trace(go.Scattergeo(
    lon=[(satellite_A_lon + satellite_B_lon) / 2],
    lat=[(satellite_A_lat + satellite_B_lat) / 2],
    text=[f'Distance: {distance:.2f} km'],
    mode='text',
    textposition='top center',
    name='Distance Annotation'
))
# Show the plot
fig.show()


In [36]:
# Calculate coordinates for the circle around each satellite
radius = 50  #50 km radius

# Calculate the latitude and longitude points for the circle
# for more accurate calculation, please fine tune your ratio, i.e. 111
circle_A_lon = satellite_A_lon + np.cos(np.linspace(0, 2 * np.pi, 100)) * (radius / 111) # 1 degree of lon is approximately 111.32 km
circle_A_lat = satellite_A_lat + np.sin(np.linspace(0, 2 * np.pi, 100)) * (radius / 111) # 1 degree of lon is approximately 111.32 km
circle_B_lon = satellite_B_lon + np.cos(np.linspace(0, 2 * np.pi, 100)) * (radius / 111) # 1 degree of lon is approximately 111.32 km
circle_B_lat = satellite_B_lat + np.sin(np.linspace(0, 2 * np.pi, 100)) * (radius / 111) # 1 degree of lon is approximately 111.32 km

# Create a scatter_geo plot
fig = go.Figure()

# Add satellite A
fig.add_trace(go.Scattergeo(
    lon=[satellite_A_lon],
    lat=[satellite_A_lat],
    mode='markers',
    marker=dict(color='blue', size=8),
    name='Satellite A'
))

# Add satellite B
fig.add_trace(go.Scattergeo(
    lon=[satellite_B_lon],
    lat=[satellite_B_lat],
    mode='markers',
    marker=dict(color='red', size=8),
    name='Satellite B'
))

# Add circles around the satellites with a radius of 50 km
fig.add_trace(go.Scattergeo(
    lon=circle_A_lon,
    lat=circle_A_lat,
    mode='lines',
    line=dict(color='blue', width=1),
    name='50 km Radius A'
))

fig.add_trace(go.Scattergeo(
    lon=circle_B_lon,
    lat=circle_B_lat,
    mode='lines',
    line=dict(color='red', width=1),
    name='50 km Radius B'
))

# Add line connecting the satellites
fig.add_trace(go.Scattergeo(
    lon=[satellite_A_lon, satellite_B_lon],
    lat=[satellite_A_lat, satellite_B_lat],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Distance'
))

# Set layout properties
fig.update_layout(
    title='Satellite Positions with 50 km Radius Circle Around',
    geo=dict(
        resolution=110,
        showland=True,
        showlakes=True,
        landcolor='rgb(204, 204, 204)',
        countrycolor='rgb(204, 204, 204)',
        lakecolor='rgb(255, 255, 255)',
        projection_type='equirectangular',
        coastlinewidth=0.5,
        lataxis=dict(range=[-90, 90]),
        lonaxis=dict(range=[-180, 180])
    ),
    legend=dict(
        x=0.5,
        y=1.08,
        orientation='h'
    ),
    margin=dict(r=0, l=0, t=60, b=0),
)

# Show the plot
fig.show()